# Autotrader
This notebook shows the basic steps to use the Coinbase API to check prices and to make trade orders. The code bases on the following YouTube video: https://www.youtube.com/watch?v=a74pQbHgdXw

It is required to create an account at coinbase and to create credentials for the API. Here, a sandbox is used which is automatically available for each account. The sandbox can be accessed here: https://public.sandboxpro.coinbase.com

Select API from the menu to find the option to create credentials.

In [1]:
%config Completer.use_jedi = False

In [2]:
# Import required libraries
import pandas as pd
import cbpro
import json


In [3]:
# Load credentials from JSON file
with open("auth_credentials.json") as json_data_file:
    auth_credentials = json.load(json_data_file)

In [4]:
# Websocket client to receive prices from Coinbase
class TextWebsocketClient(cbpro.WebsocketClient):
    
    # Open websocket connection
    def on_open(self):
        self.url = 'wss://ws-feed.pro.coinbase.com'
        self.message_count = 0
        
    # Receive a price
    def on_message(self, msg):
        self.message_count += 1
        msg_type = msg.get('type', None)
        if msg_type == 'ticker':
            time_val = msg.get('time', ('-' * 27))
            price_val = msg.get('price', None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id', None)
    
            print(f"{time_val:30} {price_val:.3f} {product_id}\tchannel type:{msg_type}")
    
    # Close connection
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [15]:
# Start websocket stream
stream = TextWebsocketClient(products=['BTC-USD'], channels=['ticker'])
stream.start()

2021-06-10T20:38:53.158613Z    35994.440 BTC-USD	channel type:ticker
2021-06-10T20:38:53.390123Z    35994.440 BTC-USD	channel type:ticker
2021-06-10T20:38:54.194004Z    35994.440 BTC-USD	channel type:ticker
2021-06-10T20:38:55.102806Z    35994.430 BTC-USD	channel type:ticker


In [16]:
# Close websocket stream
stream.close()

[WinError 10038] Ein Vorgang bezog sich auf ein Objekt, das kein Socket ist - data: None
<---Websocket connection closed--->
	Total messages: 5<---Websocket connection closed--->
	Total messages: 5



In [11]:
# Create an authenticated client based on credentials
url = 'https://api-public.sandbox.pro.coinbase.com'
client = cbpro.AuthenticatedClient(auth_credentials['api_key'],
                                   auth_credentials['api_secret'],
                                   auth_credentials['api_pass'],
                                   api_url=url)
payment_methods = client.get_payment_methods()
print(json.dumps(payment_methods, indent = 1))

[
 {
  "id": "b22911ee-ef35-5c97-bdd4-aef3f65618d9",
  "type": "fiat_account",
  "name": "GBP Wallet",
  "currency": "GBP",
  "primary_buy": false,
  "primary_sell": false,
  "allow_buy": true,
  "allow_sell": true,
  "allow_deposit": true,
  "allow_withdraw": true,
  "created_at": "2015-04-17T03:00:55Z",
  "updated_at": "2015-04-17T03:00:55Z",
  "resource": "payment_method",
  "resource_path": "/v2/payment-methods/b22911ee-ef35-5c97-bdd4-aef3f65618d9",
  "limits": {
   "buy": [
    {
     "period_in_days": 1,
     "total": {
      "amount": "63716.70",
      "currency": "GBP"
     },
     "remaining": {
      "amount": "63716.70",
      "currency": "GBP"
     }
    }
   ],
   "sell": [
    {
     "period_in_days": 1,
     "total": {
      "amount": "63716.70",
      "currency": "GBP"
     },
     "remaining": {
      "amount": "63716.70",
      "currency": "GBP"
     }
    }
   ]
  },
  "fiat_account": {
   "id": "eebff577-b756-58ec-9387-9bd2b8f4c4ea",
   "resource": "account"
  }
 },

In [13]:
# Define currency
for method in payment_methods:
    currency = method.get('currency', None)
    if currency.upper() == 'USD':
        method_id = method.get('id', None)
    elif currency is None:
        continue
        
print(f"Currency is '{currency}'\n")

Currency is 'USD'



In [20]:
# Make a deposit
client.deposit(
    amount=12345,
    currency=currency,
    payment_method_id=method_id
)

{'id': '7cf65003-7c66-4fec-9d37-b95639ec2f65',
 'amount': '12345.00000000',
 'currency': 'USD',
 'payout_at': '2015-02-18T16:54:00-08:00'}

In [14]:
# Make a market order
# Note: size -> crypto amount, instead use funds -> quote currency
# size = 1 -> e.g., 1 BitCoin, funds = 1000 -> e.g., BitCoins for 1000 USD
client.place_market_order(product_id='BTC-USD', side='buy', funds=500)

{'id': '512b2b1f-371d-4618-8dc4-ccf89a27b6f6',
 'product_id': 'BTC-USD',
 'side': 'buy',
 'stp': 'dc',
 'funds': '497.51243781',
 'specified_funds': '500',
 'type': 'market',
 'post_only': False,
 'created_at': '2021-06-11T19:04:45.180634Z',
 'fill_fees': '0',
 'filled_size': '0',
 'executed_value': '0',
 'status': 'pending',
 'settled': False}

In [18]:
# Check accounts
accounts = client.get_accounts()

for acc in accounts:
    currency = acc.get('currency')
    if currency == 'BTC':
        acc_id = acc.get('id')

acc_history = client.get_account_history(acc_id)

for hist in acc_history:
    print(json.dumps(hist, indent = 1))

"message"


In [19]:
client.get_accounts()

[{'id': 'a3f55a04-5366-491f-a914-f4cbe193aef2',
  'currency': 'BAT',
  'balance': '0.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '0',
  'profile_id': 'd6adcb62-bf72-4178-a8ac-07dc4ef727c2',
  'trading_enabled': True},
 {'id': '3e42e559-fc8f-4d85-9b3c-437d85c37f4f',
  'currency': 'BTC',
  'balance': '0.0134065900000000',
  'hold': '0.0000000000000000',
  'available': '0.01340659',
  'profile_id': 'd6adcb62-bf72-4178-a8ac-07dc4ef727c2',
  'trading_enabled': True},
 {'id': '4f1e710e-6eb4-4f45-9634-43c966db0e64',
  'currency': 'ETH',
  'balance': '0.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '0',
  'profile_id': 'd6adcb62-bf72-4178-a8ac-07dc4ef727c2',
  'trading_enabled': True},
 {'id': '6c86992a-1cfb-4bcd-b228-d24d4e306758',
  'currency': 'EUR',
  'balance': '1000.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '1000',
  'profile_id': 'd6adcb62-bf72-4178-a8ac-07dc4ef727c2',
  'trading_enabled': True},
 {'id': '579d4993-a156-4d